# AMS Cathode List Generator

## SUERC Radiocarbon Lab -  Submission Form Converter 

The following code blocks requires an absolute file path for an AMSEK Sample Submission Form file. This can be done by manually typing the file path into the code; however, a more convenient way is to use an ipywidgets.Button that incorporates elements of the tkinter GUI module. The result is a button that allows the user to select a file; the file path is then automatically passed to a dataframe and processed using pandas. 

###### Warning - The GUI allows the user to select multiple files and stores the respective paths as a list. Selecting multiple files at this stage in development may cause unusual behaviour.

The original Excel file is processed into two data frames, the first being "SampleList" which is a concatenation of all 4 sample pages with some additional formatting to remove footnotes and fill in blank cells such as group number. The elimination of unnecessary rows is done by checking each row for a Sample ID - No ID, no more row! The second Dataframe contains the front page but, at least for now, remains in raw format, this dataframe is called "df5".

#### Tasks Remaining
This block needs to extract the desired columns into a new file called Deltafile, which contains: Sample ID - $\delta$ $^{13}$C - $\sigma$. 

The dataframes should also be added to a database at some point, probably an SQLite database. SampleList will likely form the first database table "Requests". The second database table will probably be "Cathode Lists" and a third table for "Results", more tables can be added for additional dataframes. This database structure is, of course, up for debate.

Generating a Cathode List requires the assigning of "g-numbers" to track cathodes. A robust system for this needs to be implemented.  




In [55]:
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
import pandas as pd
from IPython.display import clear_output

#This section sets up the Jupyter Button Widget with the tkinter traits 
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
# The "Select File" Button functionality is defined above but the button itself is created much later 
        
#This function processes the Excel files for the AMSEK style sheets
        
def process_amsek():
    #Reads the Excel file from absolute path, returned by "Select File" button. 
    xls = pd.ExcelFile(file_button.files[0])  
    #xls = pd.ExcelFile(r'C:\Users\Thomas\Downloads\AMSEK200306BatchNine.xls')  

    # Returns Excel sheet names as list - Might be useful if the number of sheets vary. 
    x = xls.sheet_names
    # Creates raw dataframes for each sheet - As the number of sheets is fixed, single commands are ok.
    df1 = pd.read_excel(xls, 0, skiprows=1)
    df2 = pd.read_excel(xls, 1, skiprows=1)
    df3 = pd.read_excel(xls, 2, skiprows=1)
    df4 = pd.read_excel(xls, 3, skiprows=1)
    df5 = pd.read_excel(xls, 4, skiprows=1) # For some reason the "Front" page is last

    #creates a list of sheets - the "Front" sheet is not included but can be treated seperately
    sheets = [df1, df2, df3, df4]
    #Concatenate sheets into single dataframe
    SampleList = pd.concat(sheets)
    #Removes rows that do not have a Sample ID - i.e. footer text of each sheet
    SampleList = SampleList.dropna(subset=['Sample ID²'])
    #Fills in missing group data i.e. replaces NaN with whatever is above it.  
    SampleList['Gr¹'].fillna(method='ffill', inplace = True)


    #Sets Pandas display options to show full dataframes rather than truncated dataframes
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #Show the requested dataframe
    display(SampleList)
    
#The following function processes the NERC style submission forms   
def process_nerc():
    #Reads the Excel file from absolute path. Prefix path with 'r' to prevent unicode error with "\U"
    xls = pd.ExcelFile(file_button.files[0])  
    # Returns Excel sheet names as list - Might be useful if the number of sheets vary. 
    x = xls.sheet_names
    # Creates raw dataframes for each sheet - As the number of sheets is fixed, single commands are ok.
    df1 = pd.read_excel(xls, 0, skiprows=1) #Front Page
    df2 = pd.read_excel(xls, 1, skiprows=1) #Page 2
    df3 = pd.read_excel(xls, 2, skiprows=1) #Page 3
    df4 = pd.read_excel(xls, 3, skiprows=1) #Page 4
    df5 = pd.read_excel(xls, 4, skiprows=1) #Page 5

    #creates a list of sheets - the "Front" sheet is not included but can be treated seperately
    sheets = [df2, df3, df4, df5]
    #Concatenate sheets into single dataframe
    SampleList2 = pd.concat(sheets)
    #Removes rows that do not have a Sample ID - i.e. footer text of each sheet
    SampleList2 = SampleList2.dropna(subset=['Sample ID2'])
    #Fills in missing group data i.e. replaces NaN with whatever is above it.  
    SampleList2['Gr1'].fillna(method='ffill', inplace = True)

    #Sets Pandas display options to show full dataframes rather than truncated dataframes
    #pd.set_option("display.max_rows", None, "display.max_columns", None)
    #Show the requested dataframe
    display(SampleList2)

    
def process_clicked(b):
    clear_output()
    display(box)
    if menu.value == 1:
        process_amsek()
    else:
        process_nerc()

menu = widgets.Dropdown(
       options=[('AMSEK Submission Form',1), ('NERC Submission Form',2)],
       value=1,
       description='Select Form Type:')
process_button = widgets.Button(description='Process Form')
        
file_button = SelectFilesButton()


process_button.on_click(process_clicked)

box = widgets.HBox([file_button, menu, process_button])
box

,No.,Gr1,Sample ID2,Material,d13C (‰),s (d13C),Comments3,Cost code4,Project No.,Total vol CO2 (ml)5,Low C weight (mg)6,Estimated Age,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 4
0,1,Group 1,S16056,LMA NIST Oxalic Acid II,-17.697,NaN,FROM BULK S/13542. USE MASS SPEC S/16061,NS,14.6505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Group 1,S16171,LMA BULK Tiri Barley Mash,-27.105,NaN,FROM BULK S/13487. USE MASS SPEC S/16170.,NS,14.6507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Group 1,S16114,LMA BULK Belfast Cellulose,-23.632,NaN,NaN,NS,14.6508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Group 1,GR22081,Jetstream PCO7550,-28.539,NaN,10 microlitres added to combusiton tube,NF,15.1127,6.70,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Group 1,GR22082,Jetstream MOF7042,-29.946,NaN,10 microlitres added to combusiton tube,NF,15.1127,16.39,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Group 1,GR22083,Jetstream MOF7990,-29.113,NaN,10 microlitres added to combusiton tube,NF,15.1127,13.91,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Group 1,S16676,New make Spirit,-27.796,NaN,From D/5460 (New Make Spirit). Transfered from...,NF,15.1127,6.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Group 1,S16677,Kerosene,-39.365,NaN,From D/5459 (Kerosene). Transfered from D/546...,NF,15.1127,13.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Group 1,GR22090R,028 (station 13 bottle 15),1.080,NaN,NaN,NF,2110.0318,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Group 1,GR22106,215 (station 56 bottle 20),0.734,NaN,NaN,NF,2110.0318,2.16,NaN,Modern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
import pandas as pd

import ipywidgets as widgets

#Reads the Excel file from absolute path. Prefix path with 'r' to prevent unicode error with "\U"
xls = pd.ExcelFile(my_button.files[0])  
#xls = pd.ExcelFile(r'C:\Users\Thomas\Downloads\AMSEK200306BatchNine.xls')  

# Returns Excel sheet names as list - Might be useful if the number of sheets vary. 
x = xls.sheet_names
# Creates raw dataframes for each sheet - As the number of sheets is fixed, single commands are ok.
df1 = pd.read_excel(xls, 0, skiprows=1)
df2 = pd.read_excel(xls, 1, skiprows=1)
df3 = pd.read_excel(xls, 2, skiprows=1)
df4 = pd.read_excel(xls, 3, skiprows=1)
df5 = pd.read_excel(xls, 4, skiprows=1) # For some reason the "Front" page is last

#creates a list of sheets - the "Front" sheet is not included but can be treated seperately
sheets = [df1, df2, df3, df4]
#Concatenate sheets into single dataframe
SampleList = pd.concat(sheets)
#Removes rows that do not have a Sample ID - i.e. footer text of each sheet
SampleList = SampleList.dropna(subset=['Sample ID²'])
#Fills in missing group data i.e. replaces NaN with whatever is above it.  
SampleList['Gr¹'].fillna(method='ffill', inplace = True)

#uploader = widgets.FileUpload(accept="", multiple=False)
#display(uploader)

#Sets Pandas display options to show full dataframes rather than truncated dataframes
pd.set_option("display.max_rows", None, "display.max_columns", None)
#Show the requested dataframe
display(SampleList)



,No.,Gr¹,Sample ID²,Material,δ¹³C (‰),σ (δ¹³C),Comments³,Cost code⁴
0,1,Group 1,M8562,Oxalic II,-17.6,NaN,NaN,GS
1,2,Group 1,BK4950,Wood,-20.2,NaN,NaN,GS
2,3,Group 1,HA6123,Humic Acid,-28.6,NaN,NaN,GS
3,4,Group 1,GU55281,"carbonate, coral",-2.2,NaN,COM (2ml),GF
4,5,Group 1,GU55282,"carbonate, coral",-1.2,NaN,COM (2ml),GF
5,6,Group 1,GU55283,"carbonate, coral",-2.7,NaN,COM,GF
6,7,Group 1,GU55284,"carbonate, coral",-16.5,NaN,COM,GF
7,8,Group 1,GU55285,"carbonate, coral",-16.5,NaN,COM,GF
8,9,Group 1,GU55286,"carbonate, coral",-25.0,NaN,COM,GF
9,10,Group 1,GU55287,"carbonate, coral",-16.3,NaN,COM,GF


## NERC Lab -  Submission Form Converter 

Do the same as above but for the NERC style submission forms.

In [19]:
#Reads the Excel file from absolute path. Prefix path with 'r' to prevent unicode error with "\U"
xls = pd.ExcelFile(r'C:\Users\Thomas\Downloads\(B240) SUERC 2 MAY 2019-Updated.xlsx')  
# Returns Excel sheet names as list - Might be useful if the number of sheets vary. 
x = xls.sheet_names
# Creates raw dataframes for each sheet - As the number of sheets is fixed, single commands are ok.
df1 = pd.read_excel(xls, 0, skiprows=1) #Front Page
df2 = pd.read_excel(xls, 1, skiprows=1) #Page 2
df3 = pd.read_excel(xls, 2, skiprows=1) #Page 3
df4 = pd.read_excel(xls, 3, skiprows=1) #Page 4
df5 = pd.read_excel(xls, 4, skiprows=1) #Page 5

#creates a list of sheets - the "Front" sheet is not included but can be treated seperately
sheets = [df2, df3, df4, df5]
#Concatenate sheets into single dataframe
SampleList2 = pd.concat(sheets)
#Removes rows that do not have a Sample ID - i.e. footer text of each sheet
SampleList2 = SampleList2.dropna(subset=['Sample ID2'])
#Fills in missing group data i.e. replaces NaN with whatever is above it.  
SampleList2['Gr1'].fillna(method='ffill', inplace = True)

#Sets Pandas display options to show full dataframes rather than truncated dataframes
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#Show the requested dataframe
display(SampleList2)

,No.,Gr1,Sample ID2,Material,d13C (‰),s (d13C),Comments3,Cost code4,Project No.,Total vol CO2 (ml)5,Low C weight (mg)6,Estimated Age,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 4
0,1,Group 1,S16056,LMA NIST Oxalic Acid II,-17.697,NaN,FROM BULK S/13542. USE MASS SPEC S/16061,NS,14.6505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Group 1,S16171,LMA BULK Tiri Barley Mash,-27.105,NaN,FROM BULK S/13487. USE MASS SPEC S/16170.,NS,14.6507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Group 1,S16114,LMA BULK Belfast Cellulose,-23.632,NaN,NaN,NS,14.6508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Group 1,GR22081,Jetstream PCO7550,-28.539,NaN,10 microlitres added to combusiton tube,NF,15.1127,6.70,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Group 1,GR22082,Jetstream MOF7042,-29.946,NaN,10 microlitres added to combusiton tube,NF,15.1127,16.39,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Group 1,GR22083,Jetstream MOF7990,-29.113,NaN,10 microlitres added to combusiton tube,NF,15.1127,13.91,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Group 1,S16676,New make Spirit,-27.796,NaN,From D/5460 (New Make Spirit). Transfered from...,NF,15.1127,6.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Group 1,S16677,Kerosene,-39.365,NaN,From D/5459 (Kerosene). Transfered from D/546...,NF,15.1127,13.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Group 1,GR22090R,028 (station 13 bottle 15),1.080,NaN,NaN,NF,2110.0318,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Group 1,GR22106,215 (station 56 bottle 20),0.734,NaN,NaN,NF,2110.0318,2.16,NaN,Modern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
